In [2]:
import re

def parse_sos_file(sos_file_path):
    koordsys = None
    coordinates = []
    inside_coordinates = False

    with open(sos_file_path, 'r', encoding='ISO8859-10') as f:
        for line in f:
            line = line.strip()
            
            # Identify the KOORDSYS line
            if line.startswith('...KOORDSYS'):
                parts = line.split()
                if len(parts) >= 2:
                    koordsys = int(parts[1])

            # Check if we've reached the ..NØ section
            if line.startswith('..NØ'):
                inside_coordinates = True
                continue
            
            # If inside the NØ section, extract the coordinate pairs
            if inside_coordinates:
                if re.match(r'^\d+\s+\d+$', line):
                    x, y = line.split()
                    coordinates.append((float(x), float(y)))
                else:
                    # If we hit a line that doesn't match coordinates, break out
                    break

    return koordsys, coordinates

# Example usage:
sos_path = "../datasets/aalesund/1504200/200.sos"
koordsys, coords = parse_sos_file(sos_path)
print("KOORDSYS:", koordsys)
print("Extracted Coordinates:", coords)


KOORDSYS: 110
Extracted Coordinates: [(496779.0, 13727.0), (497712.0, 15418.0)]


In [3]:
import platform
platform.architecture()

('32bit', 'WindowsPE')

In [8]:
import os
from PIL import Image
import ctypes
import json



# Ensure that coords and koordsys are available from Cell 1

if len(coords) != 2:
    raise ValueError("Expected exactly two coordinate pairs (lower-left and upper-right corners).")

(x_min, y_min), (x_max, y_max) = coords

# Sort coordinates to get proper bounding box
x_min, x_max = sorted([x_min, x_max])
y_min, y_max = sorted([y_min, y_max])

# If koordsys is 110 (Ålesund), transform to EUREF89-32N using the DLL
if koordsys == 110:
    # Load and initialize the transformation
    dll_path = r"C:\Users\SivertEspelandHusebø\ScanAI\sos_coordinate_extracting\skt2_1507-1504_1.dll"
    href_file = r"C:\Users\SivertEspelandHusebø\ScanAI\sos_coordinate_extracting\HREF2018B_NN2000_EUREF89.bin"
    if not os.path.exists(dll_path):
        raise FileNotFoundError("DLL file not found.")
    if not os.path.exists(href_file):
        raise FileNotFoundError("HREF file not found.")
    
    trans_dll = ctypes.CDLL(dll_path)

    _InitSkTrans = trans_dll._InitSkTrans
    _InitSkTrans.argtypes = [ctypes.c_char_p, ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short)]
    _InitSkTrans.restype = None

    _GeoTrans = trans_dll._GeoTrans
    _GeoTrans.argtypes = [
        ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short),
        ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double),
        ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short),
        ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double), ctypes.POINTER(ctypes.c_double),
        ctypes.POINTER(ctypes.c_short)
    ]
    _GeoTrans.restype = None

    href = href_file.encode('utf-8')
    sLen = ctypes.c_short(len(href))
    sErr = ctypes.c_short(0)
    _InitSkTrans(href, ctypes.byref(sLen), ctypes.byref(sErr))
    if sErr.value != 0:
        raise RuntimeError("Failed to init transformation")

    # Ålesund-lokal -> EUREF89-32N
    # According to given info:
    # slSys1=4; slZone1=1 for Ålesund-lokal
    # slSys2=7; slZone2=32 for EUREF89-32N
    slSys1 = ctypes.c_short(4)
    slZone1 = ctypes.c_short(1)
    slSys2 = ctypes.c_short(7)
    slZone2 = ctypes.c_short(32)

    def transform_aalesund_to_euref(x, y):
        x1 = ctypes.c_double(x)
        y1 = ctypes.c_double(y)
        z1 = ctypes.c_double(0.0)
        x2 = ctypes.c_double(0.0)
        y2 = ctypes.c_double(0.0)
        z2 = ctypes.c_double(0.0)
        slErr = ctypes.c_short(0)

        _GeoTrans(
            ctypes.byref(slSys1), ctypes.byref(slZone1),
            ctypes.byref(x1), ctypes.byref(y1), ctypes.byref(z1),
            ctypes.byref(slSys2), ctypes.byref(slZone2),
            ctypes.byref(x2), ctypes.byref(y2), ctypes.byref(z2),
            ctypes.byref(slErr)
        )

        if slErr.value != 0:
            raise RuntimeError("Coordinate transformation failed.")
        return x2.value, y2.value

    # Transform both corners
    (x_min, y_min) = transform_aalesund_to_euref(x_min, y_min)
    (x_max, y_max) = transform_aalesund_to_euref(x_max, y_max)
    print("Coordinates transformed from Ålesund 110 to EUREF89-32N")

else:
    print("Coordinate system is not Ålesund 110. Using coordinates as is.")

image_path = "../datasets/aalesund/1504200/200.jpg"
img = Image.open(image_path)
width, height = img.size

pixel_size_x = (x_max - x_min) / width
pixel_size_y = (y_min - y_max) / height

top_left_x = x_min + (pixel_size_x / 2.0)
top_left_y = y_max - (abs(pixel_size_y) / 2.0)

world_file_path = os.path.splitext(image_path)[0] + ".jgw"
with open(world_file_path, 'w') as wf:
    wf.write(f"{pixel_size_x}\n")
    wf.write("0.0\n")
    wf.write("0.0\n")
    wf.write(f"{pixel_size_y}\n")
    wf.write(f"{top_left_y}\n")
    wf.write(f"{top_left_x}\n")
    
print(f"World file created: {world_file_path}")
print("Image georeferencing parameters have been written.")


RuntimeError: Failed to init transformation

In [19]:
import re
import os
from PIL import Image
import ctypes

def parse_sos_file(sos_file_path):
    """
    Parses the SOS file to extract the coordinate system and bounding coordinates.

    Args:
        sos_file_path (str): Path to the SOS file.

    Returns:
        tuple: (koordsys, coordinates) where koordsys is an integer and
               coordinates is a list of tuples [(x_min, y_min), (x_max, y_max)].
    """
    koordsys = None
    coordinates = []
    inside_coordinates = False

    with open(sos_file_path, 'r', encoding='ISO8859-10') as f:
        for line in f:
            line = line.strip()
            
            # Identify the KOORDSYS line
            if line.startswith('...KOORDSYS'):
                parts = line.split()
                if len(parts) >= 2:
                    koordsys = int(parts[1])

            # Check if we've reached the ..NØ section
            if line.startswith('..NØ'):
                inside_coordinates = True
                continue
            
            # If inside the NØ section, extract the coordinate pairs
            if inside_coordinates:
                if re.match(r'^\d+\s+\d+$', line):
                    x, y = line.split()
                    coordinates.append((float(x), float(y)))
                else:
                    # If we hit a line that doesn't match coordinates, break out
                    break

    return koordsys, coordinates

def initialize_transformation(dll_path, href_file):
    """
    Initializes the coordinate transformation using the provided DLL and HREF file.
    """
    if not os.path.exists(dll_path):
        raise FileNotFoundError("DLL file not found.")
    if not os.path.exists(href_file):
        raise FileNotFoundError("HREF file not found.")
    
    try:
        trans_dll = ctypes.CDLL(dll_path)
    except OSError as e:
        raise RuntimeError(f"Failed to load DLL: {e}")

    # Define argument and return types
    _InitSkTrans = trans_dll._InitSkTrans
    _InitSkTrans.argtypes = [ctypes.c_char_p, ctypes.POINTER(ctypes.c_short), ctypes.POINTER(ctypes.c_short)]
    _InitSkTrans.restype = None

    href = href_file.encode('utf-8')
    sLen = ctypes.c_short(len(href))
    sErr = ctypes.c_short(0)

    print("Initializing transformation...")
    try:
        _InitSkTrans(href, ctypes.byref(sLen), ctypes.byref(sErr))
    except Exception as e:
        print(f"Error during initialization: {e}")
    
    if sErr.value != 0:
        print(f"sErr value: {sErr.value}")  # Log the error code
        raise RuntimeError(f"Failed to initialize transformation. Error Code: {sErr.value}")

    return trans_dll, _InitSkTrans, trans_dll._GeoTrans


def transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2):
    """
    Transforms coordinates from Ålesund-lokal to EUREF89-32N.

    Args:
        x (float): Local easting.
        y (float): Local northing.
        _GeoTrans: Reference to the _GeoTrans function from the DLL.
        slSys1 (ctypes.c_short): Source system code.
        slZone1 (ctypes.c_short): Source zone code.
        slSys2 (ctypes.c_short): Target system code.
        slZone2 (ctypes.c_short): Target zone code.

    Returns:
        tuple: (Easting, Northing) in EUREF89-32N.
    """
    x1 = ctypes.c_double(x)
    y1 = ctypes.c_double(y)
    z1 = ctypes.c_double(0.0)
    x2 = ctypes.c_double(0.0)
    y2 = ctypes.c_double(0.0)
    z2 = ctypes.c_double(0.0)
    slErr = ctypes.c_short(0)

    _GeoTrans(
        ctypes.byref(slSys1), ctypes.byref(slZone1),
        ctypes.byref(x1), ctypes.byref(y1), ctypes.byref(z1),
        ctypes.byref(slSys2), ctypes.byref(slZone2),
        ctypes.byref(x2), ctypes.byref(y2), ctypes.byref(z2),
        ctypes.byref(slErr)
    )

    if slErr.value != 0:
        raise RuntimeError(f"Failed to initialize transformation. Error Code: {sErr.value}")

    # Swap to return (Easting, Northing) if DLL returns (Northing, Easting)
    return y2.value, x2.value

def create_world_file(image_path, x_min_trans, y_min_trans, x_max_trans, y_max_trans):
    """
    Creates a world file (.jgw) for the given image based on transformed coordinates.

    Args:
        image_path (str): Path to the JPEG image.
        x_min_trans (float): Transformed minimum easting.
        y_min_trans (float): Transformed minimum northing.
        x_max_trans (float): Transformed maximum easting.
        y_max_trans (float): Transformed maximum northing.
    """
    img = Image.open(image_path)
    width, height = img.size

    # Calculate pixel sizes
    pixel_size_x = (x_max_trans - x_min_trans) / width
    pixel_size_y = (y_min_trans - y_max_trans) / height  # Negative for north-up

    print(f"Pixel Size X: {pixel_size_x} meters/pixel")
    print(f"Pixel Size Y: {pixel_size_y} meters/pixel (negative for north-up)")

    # Assign top-left pixel coordinates without half-pixel offset
    top_left_easting = x_min_trans
    top_left_northing = y_max_trans

    print(f"Top-Left Pixel Coordinates: (Easting: {top_left_easting}, Northing: {top_left_northing})")

    # Create the world file path
    world_file_path = os.path.splitext(image_path)[0] + ".jgw"
    with open(world_file_path, 'w') as wf:
        wf.write(f"{pixel_size_x}\n")       # Line 1: Pixel size in X
        wf.write("0.0\n")                   # Line 2: Rotation about Y-axis
        wf.write("0.0\n")                   # Line 3: Rotation about X-axis
        wf.write(f"{pixel_size_y}\n")       # Line 4: Pixel size in Y (negative)
        wf.write(f"{top_left_easting}\n")   # Line 5: X coordinate of top-left pixel
        wf.write(f"{top_left_northing}\n")  # Line 6: Y coordinate of top-left pixel

    print(f"World file created: {world_file_path}")
    print("Image georeferencing parameters have been written.")

def main():
    # Define paths
    sos_path = "../datasets/aalesund/1504201/201.sos"
    image_path = "../datasets/aalesund/1504201/201.jpg"
    dll_path = r"skt2_1507-1504_1.dll"
    href_file = r"HREF2018B_NN2000_EUREF89.bin"

    # Parse the SOS file
    koordsys, coords = parse_sos_file(sos_path)
    print("KOORDSYS:", koordsys)
    print("Extracted Coordinates:", coords)

    if len(coords) != 2:
        raise ValueError("Expected exactly two coordinate pairs (lower-left and upper-right corners).")

    (x_min, y_min), (x_max, y_max) = coords

    # Sort coordinates to get proper bounding box
    x_min, x_max = sorted([x_min, x_max])
    y_min, y_max = sorted([y_min, y_max])

    print(f"Sorted Coordinates:")
    print(f"X_min: {x_min}, X_max: {x_max}")
    print(f"Y_min: {y_min}, Y_max: {y_max}")

    # Initialize transformation if KOORDSYS is 110 (Ålesund)
    if koordsys == 110:
        trans_dll, _InitSkTrans, _GeoTrans = initialize_transformation(dll_path, href_file)
        print("Transformation DLL initialized successfully.")

        # Define system and zone codes
        slSys1 = ctypes.c_short(4)   # NGO1948 / Ålesund-lokal
        slZone1 = ctypes.c_short(1)  # Axis 1
        slSys2 = ctypes.c_short(7)   # EUREF89
        slZone2 = ctypes.c_short(32) # Zone 32

        # Define the transformation function
        def transform(x, y):
            return transform_aalesund_to_euref(x, y, _GeoTrans, slSys1, slZone1, slSys2, slZone2)

        # Transform both corners
        try:
            easting_min, northing_min = transform(x_min, y_min)
            easting_max, northing_max = transform(x_max, y_max)
            print("Coordinates transformed from Ålesund 110 to EUREF89-32N")
            print(f"Transformed Bottom-Left: (Easting: {easting_min}, Northing: {northing_min})")
            print(f"Transformed Top-Right: (Easting: {easting_max}, Northing: {northing_max})")
        except RuntimeError as e:
            print(e)
            return
    else:
        print("Coordinate system is not Ålesund 110. Using coordinates as is.")
        # If not KOORDSYS=110, assume coordinates are already in EUREF89-32N
        easting_min, northing_min = x_min, y_min
        easting_max, northing_max = x_max, y_max
        print(f"Using original coordinates:")
        print(f"Bottom-Left: (Easting: {easting_min}, Northing: {northing_min})")
        print(f"Top-Right: (Easting: {easting_max}, Northing: {northing_max})")

    # Create the world file
    create_world_file(image_path, easting_min, northing_min, easting_max, northing_max)

if __name__ == "__main__":
    main()


KOORDSYS: 110
Extracted Coordinates: [(497197.0, 16073.0), (498310.0, 16976.0)]
Sorted Coordinates:
X_min: 497197.0, X_max: 498310.0
Y_min: 16073.0, Y_max: 16976.0
Initializing transformation...
Transformation DLL initialized successfully.
Coordinates transformed from Ålesund 110 to EUREF89-32N
Transformed Bottom-Left: (Easting: 364010.2971850267, Northing: 6928533.038357328)
Transformed Top-Right: (Easting: 364962.85623642476, Northing: 6929603.448842814)
Pixel Size X: 0.21083644342586796 meters/pixel
Pixel Size Y: -0.19231234018801652 meters/pixel (negative for north-up)
Top-Left Pixel Coordinates: (Easting: 364010.2971850267, Northing: 6929603.448842814)
World file created: ../datasets/aalesund/1504201/201.jgw
Image georeferencing parameters have been written.
